In [13]:
import pandas as pd
import requests
import logging
from typing import List, Optional
from requests.exceptions import RequestException

In [14]:
# Configurar logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [15]:
# Configuración de archivos y endpoints
#ruta_hired_employees = "archivos/hired_employees.csv"
ruta_hired_employees = "archivos/employees.csv"
ruta_departments = "archivos/departments.csv"
ruta_jobs = "archivos/Jobs.csv"

In [16]:
columns_names_hired_employees = ["id", "name", "datetime", "department_id", "job_id"]
columns_names_departments = ["id", "department"]
columns_names_jobs = ["id", "job"]

In [17]:
API_URL_hired_employees = "http://localhost:8000/hired_employees/batch_insert"
API_URL_deparments = "http://localhost:8000/departments/batch_insert"
API_URL_jobs = "http://localhost:8000/jobs/batch_insert"

In [18]:
def leer_csv_en_chunks(ruta: str, chunk_size: int) -> pd.io.parsers.TextFileReader:
    """Lee un archivo CSV por chunks."""
    return pd.read_csv(ruta, header=None, chunksize=chunk_size)

In [19]:
def transformar_chunk(chunk: pd.DataFrame, columnas: List[str]) -> pd.DataFrame:
    """Asigna columnas, convierte tipos y reemplaza NaN por None."""
    chunk.columns = columnas
    chunk = chunk.astype(object)
    return chunk.where(pd.notnull(chunk), None)

In [20]:
def convertir_a_json(chunk: pd.DataFrame) -> List[dict]:
    """Convierte un DataFrame a lista de diccionarios (JSON)."""
    return chunk.to_dict(orient="records")

In [21]:
def enviar_a_api(api_url: str, data: List[dict]) -> Optional[requests.Response]:
    """Envía datos en formato JSON a la API y devuelve la respuesta."""
    try:
        response = requests.post(api_url, json=data, timeout=10)
        response.raise_for_status()
        return response
    except RequestException as e:
        # Mostrar el contenido de la respuesta si está disponible
        if hasattr(e, 'response') and e.response is not None:
            logger.error(f"Error al enviar datos a la API ({api_url}): {e.response.text}")
        else:
            logger.error(f"Error al enviar datos a la API ({api_url}): {e}")
        return None

In [22]:
def procesar_y_enviar_csv(
    ruta: str,
    columnas: List[str],
    api_url: str,
    chunk_size: int = 2000
) -> None:
    """Proceso completo de lectura, transformación y envío a API."""
    for i, chunk in enumerate(leer_csv_en_chunks(ruta, chunk_size)):
        df = transformar_chunk(chunk, columnas)
        json_data = convertir_a_json(df)
        response = enviar_a_api(api_url, json_data)

        if response:
            logger.info(f"[{api_url}] Chunk {i+1} enviado con éxito ({len(json_data)} registros)")
        else:
            logger.error(f"[{api_url}] Error en el envío del chunk {i+1}")
            break  # Detener en caso de error

In [23]:
def main() -> None:
    """Función principal del proceso ETL."""
    logger.info("Iniciando carga de datos...")

    procesar_y_enviar_csv(ruta_departments, columns_names_departments, API_URL_deparments)
    procesar_y_enviar_csv(ruta_jobs, columns_names_jobs, API_URL_jobs)
    procesar_y_enviar_csv(ruta_hired_employees, columns_names_hired_employees, API_URL_hired_employees)

    logger.info("Carga de datos finalizada.")

In [25]:
if __name__ == "__main__":
    main()

2025-08-12 22:38:24,050 - INFO - Iniciando carga de datos...
2025-08-12 22:38:26,182 - INFO - [http://localhost:8000/departments/batch_insert] Chunk 1 enviado con éxito (12 registros)
2025-08-12 22:38:28,278 - INFO - [http://localhost:8000/jobs/batch_insert] Chunk 1 enviado con éxito (183 registros)
2025-08-12 22:38:30,464 - INFO - [http://localhost:8000/hired_employees/batch_insert] Chunk 1 enviado con éxito (2000 registros)
2025-08-12 22:38:32,635 - INFO - [http://localhost:8000/hired_employees/batch_insert] Chunk 2 enviado con éxito (2000 registros)
2025-08-12 22:38:34,744 - INFO - [http://localhost:8000/hired_employees/batch_insert] Chunk 3 enviado con éxito (2000 registros)
2025-08-12 22:38:36,890 - INFO - [http://localhost:8000/hired_employees/batch_insert] Chunk 4 enviado con éxito (2000 registros)
2025-08-12 22:38:39,010 - INFO - [http://localhost:8000/hired_employees/batch_insert] Chunk 5 enviado con éxito (2000 registros)
2025-08-12 22:38:41,162 - INFO - [http://localhost:800